In [1]:
import os
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [2]:
#하이퍼 파라미터 튜닝

CFG = {
    'IMG_SIZE':128, #이미지 사이즈
    'EPOCHS':50, #에포크
    'LEARNING_RATE':2e-2, #학습률
    'BATCH_SIZE':12, #배치사이즈
    'SEED':41, #시드
}

In [3]:
import random
import numpy as np

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED'])

In [4]:
import pandas as pd
label_df = pd.read_csv('/Users/kyunghee/Library/Mobile Documents/com~apple~CloudDocs/깃허브 repo/Dacon-Kaggle-practice/data/hand_dataset/train/train.csv')
label_df.head()

,file_name,label
0,001.png,10-2
1,002.png,10-1
2,003.png,3
3,004.png,8
4,005.png,9


In [5]:
label_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 858 entries, 0 to 857
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_name  858 non-null    object
 1   label      858 non-null    object
dtypes: object(2)
memory usage: 13.5+ KB


## 데이터 전처리

모델링을 하기 위해 label을 정수형으로 수정

In [6]:
label_df['label'][label_df['label']=='10-1'] = 10
label_df['label'][label_df['label']=='10-2'] = 0
label_df['label'] = label_df['label'].apply(lambda x : int(x))

In [7]:
label_df

,file_name,label
0,001.png,0
1,002.png,10
2,003.png,3
3,004.png,8
4,005.png,9
...,...,...
853,854.png,9
854,855.png,1
855,856.png,4
856,857.png,10


In [8]:
label_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 858 entries, 0 to 857
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_name  858 non-null    object
 1   label      858 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 13.5+ KB


데이터 이미지의 local adress 와 label값을 list에 저장

In [9]:
from glob import glob

def get_train_data(data_dir):
    img_path_list = []
    label_list = []

    # get image path
    img_path_list.extend(glob(os.path.join(data_dir, '*.png')))
    img_path_list.sort(key=lambda x:int(x.split('/')[-1].split('.')[0]))
    print('wow',img_path_list)

    # get label
    # label_df = pd.read_csv(data_dir+'/train.csv')
    label_list.extend(label_df['label'])
    print('wow2', label_list)

    return img_path_list, label_list

def get_test_data(data_dir):
    img_path_list = []

    # get image path
    img_path_list.extend(glob(os.path.join(data_dir, '*.png')))
    img_path_list.sort(key=lambda x:int(x.split('/')[-1].split('.')[0]))
    print(img_path_list)

    return img_path_list

In [10]:
all_img_path, all_label = get_train_data('/Users/kyunghee/Library/Mobile Documents/com~apple~CloudDocs/깃허브 repo/Dacon-Kaggle-practice/data/hand_dataset/train')
test_img_path = get_test_data('/Users/kyunghee/Library/Mobile Documents/com~apple~CloudDocs/깃허브 repo/Dacon-Kaggle-practice/data/hand_dataset/test')

wow ['/Users/kyunghee/Library/Mobile Documents/com~apple~CloudDocs/깃허브 repo/Dacon-Kaggle-practice/data/hand_dataset/train/001.png', '/Users/kyunghee/Library/Mobile Documents/com~apple~CloudDocs/깃허브 repo/Dacon-Kaggle-practice/data/hand_dataset/train/002.png', '/Users/kyunghee/Library/Mobile Documents/com~apple~CloudDocs/깃허브 repo/Dacon-Kaggle-practice/data/hand_dataset/train/003.png', '/Users/kyunghee/Library/Mobile Documents/com~apple~CloudDocs/깃허브 repo/Dacon-Kaggle-practice/data/hand_dataset/train/004.png', '/Users/kyunghee/Library/Mobile Documents/com~apple~CloudDocs/깃허브 repo/Dacon-Kaggle-practice/data/hand_dataset/train/005.png', '/Users/kyunghee/Library/Mobile Documents/com~apple~CloudDocs/깃허브 repo/Dacon-Kaggle-practice/data/hand_dataset/train/006.png', '/Users/kyunghee/Library/Mobile Documents/com~apple~CloudDocs/깃허브 repo/Dacon-Kaggle-practice/data/hand_dataset/train/007.png', '/Users/kyunghee/Library/Mobile Documents/com~apple~CloudDocs/깃허브 repo/Dacon-Kaggle-practice/data/hand_dat

In [11]:
all_label[0:5]

[0, 10, 3, 8, 9]

# CustomDataset

In [12]:
import torchvision.datasets as datasets 
import torchvision.transforms as transforms
import cv2
from torch.utils.data import DataLoader, Dataset

class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, train_mode = True, transforms = None):
        self.transforms = transforms
        self.train_mode = train_mode
        self.img_path_list = img_path_list
        self.label_list = label_list

    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        # get image data
        image = cv2.imread(img_path)
        if self.transforms is not None:
            image = self.transforms(image)

        if self.train_mode:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

### Train/Validation Split

In [13]:
train_len = int(len(all_img_path)*0.75)
vali_len = int(len(all_img_path)*0.25)

train_img_path = all_img_path[:train_len]
train_label = all_label[:train_len]

vali_img_path = all_img_path[train_len:]
vali_label = all_label[train_len:]

In [14]:
print('train set 길이 : ', train_len)
print('validation set 길이 : ', vali_len)

train set 길이 :  643
validation set 길이 :  214


In [15]:
train_transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize([CFG['IMG_SIZE'], CFG['IMG_SIZE']]), # 이미지 사이즈 변형
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5)) # 이미지 정규화
])

test_transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize([CFG['IMG_SIZE'], CFG['IMG_SIZE']]), # 이미지 사이즈 변형
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5)) # 이미지 정규화
])

# DataLoader

데이터 로더를 통해 데이터셋의 전체 데이터가 배치 사이즈로 나뉘게 된다.
만들었던 데이터셋을 인풋으로 넣어주면 여러 옵션(데이터 묶기, 섞기, 알아서 병렬처리)을 통해 batch를 만들어냄

In [16]:
# get data loader 

train_dataset = CustomDataset(train_img_path, train_label, train_mode = True, transforms= train_transforms)
# 배치 만들기
train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

# validation에서도 적용
vali_dataset = CustomDataset(vali_img_path, vali_label, train_mode = True, transforms=test_transforms)
vali_loader = DataLoader(vali_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [17]:
train_batches = len(train_loader)
vali_batches = len(vali_loader)

print('total train imgs : ',train_len, '/ total train batches : ', train_batches)
print('total valid imgs : ',vali_len, '/ total valid batches : ', vali_batches)

total train imgs :  643 / total train batches :  54
total valid imgs :  214 / total valid batches :  18


## 모델 불러오기 

In [18]:
from torchvision import models
from torchvision.models import efficientnet_b3 as efficientnet

model = models.efficientnet_b3(pretrained=False)

/Users/kyunghee/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/kyunghee/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [19]:
model.classifier

Sequential(
  (0): Dropout(p=0.3, inplace=True)
  (1): Linear(in_features=1536, out_features=1000, bias=True)
)

## 모델학습

In [20]:
import torch.optim as optim

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = None

In [21]:
from tqdm import tqdm



def train(model, optimizer, train_loader, scheduler, device):
    model.to(device)
    n = len(train_loader)

    ## LOSS FUNCTION 정의 
    best_acc = 0

    for epoch in range(1, CFG["EPOCHS"]+1):
        model.train()
        running_loss = 0.0

        for img, label in tqdm(iter(train_loader)):
            img, label = img.to(device), label.to(device)
            optimizer.zero_grad()

            # data -> model -> output
            logit = model(img)
            loss = criterion(logit, label)

            # 역전파
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print('[%d] train loss : %.10f' % (epoch, running_loss/len(train_loader)))


        if scheduler is not None:
            scheduler.step()

        # validation 평가
        model.eval()
        vali_loss = 0.0
        correct = 0
        with torch.no_grad():
            for img, label in tqdm(iter(vali_loader)):
                img, label = img.to(device), label.to(device)

                logit = model(img)
                vali_loss += criterion(logit, label)
                pred = logit.argmax(dim=1, keepdim=True)
                correct += pred.eq(label.view_as(pred)).sum().item()
        vali_acc = 100 * correct /len(vali_loader.dataset)
        print('Vail set: Loss: {:.4f}, Accuracy: {}/{} ( {:.0f}%)\n'.format(vali_loss / len(vali_loader), correct, len(vali_loader.dataset), 100* correct/len(vali_loader.dataset)))

        # best model 저장
        if best_acc < vali_acc:
            best_acc = vali_acc
            torch.save(model.state_dict(), './saved/best_model.pth')
            print('model saved.')

In [22]:
train(model, optimizer, train_loader, scheduler, device)

  0%|          | 0/54 [00:00<?, ?it/s]

100%|██████████| 54/54 [02:56<00:00,  3.28s/it]


[1] train loss : 4.0073316186


100%|██████████| 18/18 [00:16<00:00,  1.09it/s]


Vail set: Loss: 17.1208, Accuracy: 19/215 ( 9%)



RuntimeError: Parent directory ./saved does not exist.